In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import geopandas as gpd
import requests

from zipfile import ZipFile

from shapely.geometry import Point, Polygon

In [5]:
def konwersja_wspolrzedne(stopnie, minuty, sekundy, kierunek):
    wspolrzedne = stopnie + (minuty / 60) + (sekundy / 3600)
    if kierunek in ['W', 'S']:
        wspolrzedne = -decimal_degrees
    return wspolrzedne

#### TERYT 

In [6]:
TERYT = pd.read_csv('TERC_Urzedowy_2025-07-03/TERC_Urzedowy_2025-07-03.csv', sep=';', dtype={'WOJ': 'str', 'POW': 'str', 'GMI': 'str', 'RODZ': 'str'})

In [7]:
TERYT['ID_WOJ'] = TERYT['WOJ']
TERYT['ID_POW'] = TERYT['WOJ']+TERYT['POW']
TERYT['ID_GMI'] = TERYT['WOJ']+TERYT['POW']+TERYT['GMI']+TERYT['RODZ']

In [8]:
WOJ = TERYT[TERYT['POW'].isnull()]
WOJ = WOJ.drop(columns={'ID_POW', 'ID_GMI','WOJ', 'POW', 'GMI', 'RODZ', 'STAN_NA'})
POW = TERYT[(TERYT['GMI'].isnull()) & (TERYT['POW'].notnull())]
POW = POW.drop(columns={'WOJ', 'POW', 'GMI', 'RODZ', 'STAN_NA', 'ID_GMI'})
GMI = TERYT[(TERYT['GMI'].notnull()) & (TERYT['RODZ'].notnull())]
GMI = GMI.drop(columns={'WOJ','POW', 'GMI', 'STAN_NA', 'ID_WOJ'})

In [9]:
WOJ.to_csv('WOJ.csv', index=False, sep=',', encoding='Windows 1250')
POW.to_csv('POW.csv', index=False, sep=',', encoding='Windows 1250')
GMI.to_csv('GMI.csv', index=False, sep=',', encoding='Windows 1250')

#### DANE 

In [10]:
## ID, NAZWA ORAZ KOD STACJI
WYKAZ_STACJI = pd.read_csv('wykaz_stacji.csv', encoding='Windows 1250', header=None, names=['ID', 'NAZWA', 'KOD'], dtype={'KOD': 'object', 'ID': 'object'})

WYKAZ_STACJI['KOD'] = WYKAZ_STACJI['KOD'].str.strip()

In [11]:
# WSPOLRZEDNE GPS STACJI WRAZ KODEM I NAZWA
STACJE = pd.read_excel("STACJE.xlsx", header=None)
STACJE.columns = ['KOLUMNA']

In [12]:
# WYCIĄGNIECIE WSPOLRZEDNYCH - AKTUALNY FORMAT SIE NIE NADAWAL
STACJE["KOD"] = STACJE['KOLUMNA'].str.split(' ').str[0]
STACJE["NAZWA"] = STACJE['KOLUMNA'].str.split(' ').str[1]

STACJE["_1"] = STACJE['KOLUMNA'].str.split(' ').str[2]
STACJE["_2"] = STACJE['KOLUMNA'].str.split(' ').str[3]
STACJE["_3"] = STACJE['KOLUMNA'].str.split(' ').str[4]
STACJE["_4"] = STACJE['KOLUMNA'].str.split(' ').str[5]
STACJE["_5"] = STACJE['KOLUMNA'].str.split(' ').str[6]
STACJE["_6"] = STACJE['KOLUMNA'].str.split(' ').str[7]

STACJE['D1'] = STACJE['_1'].astype(int)
STACJE['D2'] = STACJE['_2'].astype(int)
STACJE['D3'] = np.where(STACJE['_5'].str.isdigit(), STACJE['_3'], '0').astype(int)

STACJE['S1'] = np.where(STACJE['_5'].str.isdigit(), STACJE['_4'], STACJE['_3']).astype(int)
STACJE['S2'] = np.where(STACJE['_5'].str.isdigit(), STACJE['_5'], STACJE['_4']).astype(int)
STACJE['S3'] = np.where(STACJE['_5'].str.isdigit(), STACJE['_6'], '0').astype(int)

# STACJE[~STACJE['D1'].apply(lambda x: str(x).isdigit())]

STACJE['LAT'] = konwersja_wspolrzedne(STACJE['S1'], STACJE['S2'], STACJE['S3'], 'N')
STACJE['LON'] = konwersja_wspolrzedne(STACJE['D1'], STACJE['D2'], STACJE['D3'], 'N')

In [13]:
LISTA_STACJE = (WYKAZ_STACJI.merge(STACJE[['KOD', 'LAT', 'LON']], 'left', on='KOD', indicator=True)
                .loc[lambda x: x['_merge'] == 'both']
                .reset_index(drop=True)
                .drop(columns=['_merge'])
               )


#### SHAPEFILE

In [16]:
shp_w = gpd.read_file('00_jednostki_administracyjne/A01_Granice_wojewodztw.shp')
shp_p = gpd.read_file('00_jednostki_administracyjne/A02_Granice_powiatow.shp')
shp_g = gpd.read_file('00_jednostki_administracyjne/A03_Granice_gmin.shp')

In [17]:
for stacja in range(LISTA_STACJE.shape[0]):
    # print(stacja)
    point = Point(tuple(LISTA_STACJE.loc[stacja][['LON', 'LAT']]))
    for wojewodztwo in range(shp_w.shape[0]):
        wojewodztwo_teryt = shp_w.loc[wojewodztwo, 'JPT_KOD_JE']
        polygon = shp_w.loc[wojewodztwo, 'geometry']
        czy_w_srodku = polygon.contains(point)
        if czy_w_srodku == True:
            # print('TAK')
            LISTA_STACJE.loc[stacja, 'ID_WOJ'] = wojewodztwo_teryt
    for powiat in range(shp_p.shape[0]):
        powiat_teryt = shp_p.loc[powiat, 'JPT_KOD_JE']
        polygon = shp_p.loc[powiat, 'geometry']
        czy_w_srodku = polygon.contains(point)
        if czy_w_srodku == True:
            # print('TAK')
            LISTA_STACJE.loc[stacja, 'ID_POW'] = powiat_teryt
    for gmina in range(shp_g.shape[0]):
        gmina_teryt = shp_g.loc[gmina, 'JPT_KOD_JE']
        polygon = shp_g.loc[gmina, 'geometry']
        czy_w_srodku = polygon.contains(point)
        if czy_w_srodku == True:
            # print('TAK')
            LISTA_STACJE.loc[stacja, 'ID_GMI'] = gmina_teryt

C:\Users\drynkiewicz\AppData\Local\Temp\ipykernel_20184\1002392590.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '08' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  LISTA_STACJE.loc[stacja, 'ID_WOJ'] = wojewodztwo_teryt
C:\Users\drynkiewicz\AppData\Local\Temp\ipykernel_20184\1002392590.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0809' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  LISTA_STACJE.loc[stacja, 'ID_POW'] = powiat_teryt
C:\Users\drynkiewicz\AppData\Local\Temp\ipykernel_20184\1002392590.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0809013' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  LISTA_STACJE.loc[stacja, 'I

In [43]:
## stacja w darłowie znajduje się na morzu poza zakresem shapefile
LISTA_STACJE = LISTA_STACJE.fillna({'ID_WOJ': '32', 'ID_POW': '3213', 'ID_GMI': '3213011'})

In [19]:
(LISTA_STACJE
 .to_csv('LISTA_STACJE.csv', index=False, sep=',', encoding='Windows 1250')
)

#### DANE METEO 

In [12]:
for rok in [str(x) for x in np.arange(2001, 2026, 1)]:
    for msc in ["01", "02", "03", "04" ,"05",'06',"07",'08','09','10','11','12']:
        LINK = f"https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/{rok}/{rok}_{msc}_k.zip"
        PLIK = requests.get(LINK).content

        with open(f"{rok+"_"+msc}.zip", "wb") as file:
            file.write(PLIK)
    print(rok)


2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025


In [15]:
for plik in os.listdir('PLIKI_NOWE/'):
    # print(plik)
    with ZipFile(f'PLIKI_NOWE/{plik}', 'r') as plik_zip:
        try:
            plik_zip.extractall('C:\\PythonProjects\\meteo\\PLIKI_CSV')
        except:
            print(plik)
            print('blad')
            pass
    

2024_07.zip
blad


In [3]:
df = pd.DataFrame()
for plik in os.listdir('PLIKI_CSV/'):
    if plik[:5] != 'k_d_t':
        df_temp = pd.read_csv(f'PLIKI_CSV/{plik}', encoding='Windows 1250', header=None, usecols=[0,1,2,3,4,5,7,9, 13], names=['ID', 'NAZWA', 'ROK', 'MIESIAC', 'DZIEN', 'TEMP_MAX', 'TEMP_MIN', 'TEMP_MEAN', 'PREC'], dtype={'ID': 'object'})
        
        df = pd.concat([df, df_temp], axis=0, ignore_index=True)

In [77]:
df.drop(columns=['NAZWA']).to_csv('DANE_METEO.csv', index=False, sep=',', encoding='Windows 1250')

#### SQL 

In [60]:
conn = sqlite3.connect("METEO_DB.sqlite")
cursor = conn.cursor()

In [61]:
cursor.execute("""CREATE TABLE WOJ (
    ID_WOJ   TEXT PRIMARY KEY,
    NAZWA TEXT    NOT NULL,
    NAZWA_DOD      TEXT NOT NULL);
    """)

cursor.execute("""CREATE TABLE POW (
    ID_POW   TEXT PRIMARY KEY,
    NAZWA TEXT    NOT NULL,
    NAZWA_DOD      TEXT NOT NULL,
    ID_WOJ TEXT NOT NULL,
    FOREIGN KEY (ID_WOJ)
       REFERENCES WOJ (ID_WOJ) );
    """)

cursor.execute("""CREATE TABLE GMI (
    ID_GMI   TEXT PRIMARY KEY,
    NAZWA TEXT    NOT NULL,
    NAZWA_DOD      TEXT NOT NULL,
    RODZ TEXT NOT NULL,
    ID_POW TEXT NOT NULL,
    FOREIGN KEY (ID_POW)
       REFERENCES POW (ID_POW) );
    """)

cursor.execute("""CREATE TABLE STACJE (
    ID_STACJA   TEXT PRIMARY KEY,
    NAZWA TEXT    NOT NULL,
    KOD      TEXT NOT NULL,
    LAT REAL NOT NULL,
    LON REAL NOT NULL,
    ID_WOJ TEXT NOT NULL,
    ID_POW TEXT NOT NULL,
    ID_GMI TEXT NOT NULL,
    FOREIGN KEY (ID_WOJ) REFERENCES WOJ (ID_WOJ),
    FOREIGN KEY (ID_POW) REFERENCES POW (ID_POW),
    FOREIGN KEY (ID_GMI) REFERENCES GMI (ID_GMI)
    );
    """)

cursor.execute("""CREATE TABLE METEO (
    METEO_ID   INTEGER PRIMARY KEY AUTOINCREMENT,
    ID_STACJA TEXT    NOT NULL,
    ROK      TEXT NOT NULL,
    MIESIAC TEXT NOT NULL,
    DZIEN TEXT NOT NULL,
    TEMP_MAX REAL NOT NULL,
    TEMP_MIN REAL NOT NULL,
    TEMP_MEAN REAL NOT NULL,
    PREC REAL NOT NULL,
    FOREIGN KEY (ID_STACJA) REFERENCES STACJE (ID_STACJA)
    );
    """)

In [24]:
conn = sqlite3.connect("METEO_DB.sqlite")
cursor = conn.cursor()

In [62]:
WOJ.to_sql('WOJ', conn, if_exists='append', index=False)
POW.to_sql('POW', conn, if_exists='append', index=False)
GMI.to_sql('GMI', conn, if_exists='append', index=False)

3952

In [63]:
LISTA_STACJE.rename(columns={'ID': 'ID_STACJA'}).to_sql('STACJE', conn, if_exists='append', index=False)

339

In [64]:
df.fillna(-1000).rename(columns={'ID': "ID_STACJA"}).drop(columns=['NAZWA']).to_sql('METEO', conn, if_exists='append', index=False)

1054042